## Step 1: Query [Crystallographic Open Database](http://www.crystallography.net)

For the queries please adhere to the Hill notation:
    
    Number of carbon atoms in a molecule is indicated first, the number of hydrogen atoms next, 
    and then the number of all other chemical elements subsequently, in alphabetical order of the
    chemical symbols. When the formula contains no carbon, all the elements, including hydrogen, 
    are listed alphabetically.

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.calculation import Calculation

from aiida.tools.dbimporters.plugins.cod import CodDbImporter

import ase.io
import numpy as np
import ipywidgets as ipw
from base64 import b64decode
from IPython.display import display, clear_output, Image, HTML
from fileupload import FileUploadWidget

import nglview

In [ ]:
atoms = None
structures = [("select structure",{"status":False})]

layout = ipw.Layout(width="400px")
style = {"description_width":"150px"}

viewer = nglview.NGLWidget()
clear_output()

In [ ]:
def query(idn=None,formula=None):
    importer = CodDbImporter()
    if idn is not None:
        return importer.query(id=idn)
    elif formula is not None:
        return importer.query(formula=formula)
        

def on_click_query(b):
    global inp_elements, structures
    structures = [("select structure",{"status":False})]
    idn = None
    formula = None
    try: 
        idn = int(inp_elements.value)
    except:
        formula = str(inp_elements.value)
    
    count = 0
    query_message.value = "Quering the database ... "
    for entry in query(idn=idn, formula=formula):
        try:
            entry_cif = entry.get_cif_node()
            formula = entry_cif.get_ase().get_chemical_formula()
        except:
            continue
        url = entry.source['uri']
        idn = entry.source['id']
        entry_name = "{} (id: {})".format(formula, idn)
        entry_add = (entry_name,
                        {
                            "status": True,
                            "cif": entry_cif,
                            "url": url,
                            "id": idn,
                        }
                    )
        structures.append(entry_add)
        count += 1
    
    query_message.value = "Quering the database ... %d structures found" % count
    drop_structure.options = structures
    if len(structures) > 1:
        drop_structure.value = structures[1][1]    

In [ ]:
inp_elements = ipw.Text(description="", value="", placeholder='e.g.: Ni Ti or id number', layout=layout, style=style)
btn_query = ipw.Button(description='Query in CoD')
btn_query.on_click(on_click_query)
query_message = ipw.HTML("Waiting for input...")

display(ipw.HBox([btn_query, inp_elements]), query_message)


## Step 2: Select Structure

In [ ]:
def refresh_structure_view():
    global viewer, atoms
    if hasattr(viewer, "component_0"):
        viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)

    viewer.add_component(nglview.ASEStructure(atoms.get_ase())) # adds ball+stick
    viewer.add_unitcell()
    viewer.center()

In [ ]:
def on_change(c):
    global atoms
    indx = c['owner'].index
    new_element = c['new']
    if new_element['status'] is False:
        return
    atoms = new_element['cif']
    formula = atoms.get_ase().get_chemical_formula()
    #print (indx, atoms)
    #formula = atoms.get_chemical_formula()
    
    # search for existing structures
    qb = QueryBuilder()
    qb.append(StructureData)
    qb.append(Calculation, filters={'extras.formula':formula}, descendant_of=StructureData)
    qb.order_by({Calculation:{'ctime':'desc'}})
    for n in qb.iterall():
        calc = n[0]
        print("Found existing calculation: PK=%d | %s"%(calc.pk, calc.get_extra("structure_description")))
        thumbnail = b64decode(calc.get_extra("thumbnail"))
        display(Image(data=thumbnail))
    struct_url = new_element['url'].split('.cif')[0]+'.html'
    link.value='<a href="{}" target="_blank">COD entry {}</a>'.format(struct_url, new_element['id'])
    refresh_structure_view()


    
drop_structure = ipw.Dropdown(description="", options=structures, style=style, layout=layout )
drop_structure.observe(on_change, names='value')
link = ipw.HTML("Link to the web-page will appear here")
display(drop_structure, link, ipw.VBox([viewer]))

## Step 3: Store in AiiDA Database

In [ ]:
def on_click_store(b):
    global atoms
    with store_out:
        clear_output()
        if atoms is None:
            print ("Specify a structure first!")
            return
        #AiiDA requires structures to have cell
#       if np.all(atoms.cell == 0.0):
#           atoms.center(vacuum=0.1)
        if data_format.value is 'CifData':
            s=atoms.copy()
        elif data_format.value is 'StructureData':
            s = StructureData(ase=atoms.get_ase())
            # ensure that tags got correctly translated into kinds 
            for t1, k in zip(atoms.get_ase().get_tags(), s.get_site_kindnames()):
                t2 = int(k[-1]) if k[-1].isnumeric() else 0
                assert t1==t2
            s.description = inp_descr.value
        
        s.store()
        print("Stored in AiiDA: "+repr(s))

inp_descr = ipw.Text(placeholder="Description (optional)")   
btn_store = ipw.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)
data_format = ipw.RadioButtons(
    options=['CifData', 'StructureData'],
#     value='pineapple',
    description='Data type:',
    disabled=False
)


store_out = ipw.Output()
display(data_format, ipw.HBox([btn_store, inp_descr]), store_out)